# Setup

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import stats

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, RepeatedKFold, cross_val_score
import sklearn.metrics as skm
from sklearn.metrics import accuracy_score
from sklearn.cluster import AffinityPropagation as AP
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import LogisticRegression

import re
import time
import datetime as dt

import sys
import os
sys.path.append(os.path.abspath
                (os.path.join
                 (os.path.dirname("constants.py"), '..')))
from constants import *
from scripts import *

import warnings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

hide_toggle()

# FUNCTIONS

In [6]:
def train_test_split_logistic(dfToSplitFunc, tournamentId, currentTarget, cut_off_var, splitType, verbose=True):
    
    for col in dfToSplitFunc.columns:
            dfToSplitFunc[col] = dfToSplitFunc[col].fillna(0)
    
    if splitType==0:
        testData = dfToSplitFunc[dfToSplitFunc['tournament_id']==tournamentId].copy()
        xtest= testData.drop(['Date',currentTarget],axis=1).copy()
        xtest= xtest.drop(offCols,axis=1,errors='ignore')
        ytest = testData[currentTarget]

        trainData = dfToSplitFunc[dfToSplitFunc['tournament_id']!=tournamentId].copy()
        xtrain = trainData.drop(['Date',currentTarget],axis=1).copy()
        xtrain = xtrain.drop(offCols,axis=1,errors='ignore')
        ytrain = trainData[currentTarget]
        
    elif splitType==1:
        #print(dfToSplitFunc.columns)
        xCols = dfToSplitFunc.drop(['Date', currentTarget]+offCols,axis=1,errors='ignore')
        #print(xCols.columns)
        yCols = dfToSplitFunc[currentTarget]
        xtrain, xtest, ytrain, ytest = train_test_split(xCols, yCols, test_size=0.20, shuffle=False)
    
    if list(ytrain).count(0)/len(ytrain)==1:
        print(len(ytrain))
        print(dropTypeF)
        print(dfToSplitFunc[currentTarget])
        print('==========================================================')
    
    ytrain_mean, ytrain_std = np.mean(ytrain), np.std(ytrain)
    cut_off = ytrain_std * cut_off_var
    lower, upper = ytrain_mean - cut_off, ytrain_mean + cut_off
    
    outlierMask = ytrain.apply(lambda x: False if x < lower or x > upper else True)
    
    if verbose:
        print(f'train len: {len(xtrain)}')
    lentemp = len(xtrain)
    #xtrain, ytrain = xtrain[outlierMask], ytrain[outlierMask]
    if verbose:
        print(f'train len no outliers: {len(xtrain)}')
        print(f'percent of len removed: {round(abs(len(xtrain)/lentemp*100-100),2)}%')
        print(f'test len: {len(xtest)}\n')
    
    return xtrain, ytrain, xtest, ytest

In [7]:
def matchListToDfs(df):
    matchListDateFilter = (df[df['Date'] >= pd.to_datetime('2019-7-01',format='%Y-%m-%d')]
                                        .reset_index(drop=True).copy())
    matchListDateFilter['realSemesterYear'] = (matchListDateFilter['realYear'].astype(str)
                                               +matchListDateFilter['realSemester'].astype(str))
    matchListDateFilter['tournament_id'] = (matchListDateFilter['TournamentRegion'].astype(str)
                                               +matchListDateFilter['realSemesterYear'].astype(str))
    
    playerMatchList = matchListDateFilter.copy()
    teamMatchList = matchListDateFilter.copy()

    for color in ['Blue','Red']:
        for feature in meanFeatures:
                teamMatchList[f'Team_{color}_{feature}'] = (matchListDateFilter[[f"{position}_{color}_{feature}" for position in positions]]
                                                        .mean(skipna=True,axis=1).copy())
                teamMatchList.drop([f"{position}_{color}_{feature}" for position in positions],axis=1,inplace=True)

        for feature in sumFeatures:
                teamMatchList[f'Team_{color}_{feature}'] = (matchListDateFilter[[f"{position}_{color}_{feature}" for position in positions]]
                                                        .sum(skipna=True,axis=1).copy())
                teamMatchList.drop([f"{position}_{color}_{feature}" for position in positions],axis=1,inplace=True)

        teamMatchList.drop([f"{position}_{color}" for position in positions],axis=1,inplace=True)
        
    return playerMatchList, teamMatchList

def regionLists(df, currentYear):
    regions = df['TournamentRegion'].unique()
    regionsToFeed = [x for x in df['TournamentRegion'].unique()]
    regionsFilterTemp = ([x for x in regions if currentYear in (df[df['TournamentRegion']==x])['realYear'].unique()
                                            and currentYear-1 in (df[df['TournamentRegion']==x])['realYear'].unique()])
    regionsToPredict = []
    for region in regionsFilterTemp:
        regionsFilterSize = df[(df['realYear']==currentYear) & (df['TournamentRegion']==region)]
        regionsFilterSizeTrain = df[(df['realYear']!=currentYear) & (df['TournamentRegion']==region)]
        if len(regionsFilterSize)>=30:
            regionsToPredict.append(region)
    
    return regions, regionsToFeed, regionsToPredict

def generateRegionDf(df, regionDataListF, regionsFeatureColsF, cut_off_var, tempTournamentIdF, currentTarget, splitType):
    
    dfTemp = df[df['TournamentRegion'].isin(regionDataListF)].copy()
    tempCols = [x for x in list(dfTemp.columns) if x.split('_')[-1] in regionsFeatureColsF]
    dfTemp = dfTemp[tempCols+infoCols]
    dfTemp = dfTemp.sort_values(by='Date',ascending=True).copy()
    
    xtrain,ytrain,xtest,ytest = train_test_split_logistic(dfTemp, tempTournamentIdF, currentTarget, cut_off_var, splitType, verbose=False)
    
    return dfTemp, xtrain, ytrain, xtest, ytest

def generateMetric(model_number, regionDataListF, regionsFeatureColsF, cut_off_var, tempTournamentIdF, currentTarget, dfToSplit, splitType):
    
    dfTemp, xtrain, ytrain, xtest, ytest = generateRegionDf(dfToSplit, regionDataListF, regionsFeatureColsF
                                                            , cut_off_var, tempTournamentIdF, currentTarget, splitType)
    
    region_model = base_models[model_number]
    region_model.fit(xtrain, ytrain)
    
    errors3=0
    rep=3
    for i in range(rep):
        region_model.fit(xtrain, ytrain)
        pred = region_model.predict_proba(xtest)
        pred = pred[:,1]
        
        errors3 = skm.mean_absolute_error(ytest, pred)+errors3
    errors2=errors3/rep

    errors3=0
    for i in range(rep):
        region_model.fit(xtrain, ytrain)
        pred = region_model.predict_proba(xtest)
        pred = pred[:,1]
    
        errors3 = skm.mean_absolute_error(ytest, pred)+errors3
    errors4=errors3/rep
    
    metric=round((errors2+errors4)/2,3)
    
    return metric, pred

hide_toggle()

# Load

In [8]:
teamDataTable = pd.read_pickle("Data/raw_data/teamDataTable.pkl")
playerDataTable = pd.read_pickle("Data/raw_data/playerDataTable.pkl")

matchList = pd.read_pickle("Data/raw_data/matchList.pkl")
matchListFill = pd.read_pickle("Data/raw_data/matchListFill.pkl")

teamMatchList = pd.read_pickle("Data/raw_data/teamMatchList.pkl")
playerMatchList = pd.read_pickle("Data/raw_data/playerMatchList.pkl")

regionsStats = pd.read_pickle("./Data/raw_data/regionsStats.pkl")

with open(f'./Data/raw_data/regionsFeatureCols.json', 'r') as fp:
    regionsFeatureCols = json.load(fp)
with open(f'./Data/raw_data/regionsTrainData.json', 'r') as fp:
    regionsTrainData = json.load(fp)
    

In [9]:
content=1
playerMatchList, teamMatchList = matchListToDfs(matchListFill)
#             0              1
dfsContent = [playerMatchList, teamMatchList]
dfToSplit = dfsContent[content].copy()

print(f'main df size: {len(dfToSplit)}')

base_models = [
              LogisticRegression(solver='newton-cg') #0
              ]

currentTarget = 'Score'
currentYear = 2022
currentSemester = 1
currentSemesterYear = str(currentYear)+str(currentSemester)
defaultModel = 0
infoCols = ['Date','tournament_id',currentTarget,'TournamentRegion']
splitType = 0

if currentTarget == 'Score':
    dfToSplit.drop('totalKills',axis=1,inplace=True)
elif currentTarget == 'totalKills':
    dfToSplit.drop('Score',axis=1,inplace=True)
    
featureCols = [x for x in dfToSplit.columns if x not in offCols+infoCols]
featureCols = list(set([x.replace('Team_Blue_','').replace('Team_Red_','') for x in featureCols]))

regions, regionsToFeed, regionsToPredict = regionLists(dfToSplit, currentYear)

dfToSplit = dfToSplit[dfToSplit['realSemesterYear'].astype(int)<=int(currentSemesterYear)]

regionsToPredict = regionsToPredict[:5]

main df size: 26314


# SKLEARN

In [10]:
regionsStats = pd.DataFrame(columns=['region','model','size'])
regionsStats['region'] = regionsToPredict
regionsList = regionsStats['region']
regionsStats['model'] = defaultModel

regionsTrainData = dict(zip(regionsList,regionsList.apply(lambda x: [x])))
regionsStats['accuracy_0'] = np.nan
regionsStats['accuracy_1'] = np.nan
regionsStats['accuracy_2'] = np.nan
regionsStats['cut_off_var'] = 1.5

regionsFeatureCols = dict(zip(regionsList,[0]*len(regionsList)))
for key in regionsFeatureCols:
    regionsFeatureCols[key] = featureCols.copy()

### TRAIN DATA SELECTION

In [11]:
%%time

regionsTrainData = dict(zip(regionsList,regionsList.apply(lambda x: [x])))
regionsStats['accuracy_0'] = np.nan

for n,region in enumerate(regionsToPredict):
    print('=========\n')
    regionFinalAcc = regionsStats['accuracy_0'][n]
    tempTournamentId = region+currentSemesterYear
    cut_off_var = regionsStats['cut_off_var'][n]
    region_model_number = regionsStats['model'][n]
    print(f'[{n+1} of {len(regionsToPredict)}] region {region} -> {regionFinalAcc}:\n')
    
    regionsToTest = [x for x in regionsToFeed]
    regionsToTest.remove(region)
    random.shuffle(regionsToTest)
    for nn,regionToTest in enumerate(regionsToTest):
        regionsTrainData[region].append(regionToTest)
        
        metric, pred = generateMetric(region_model_number, regionsTrainData[region], regionsFeatureCols[region]
                                      , cut_off_var, tempTournamentId, currentTarget, dfToSplit, splitType)
        
        if metric < regionFinalAcc or np.isnan(regionFinalAcc):
            regionFinalAcc = metric
            
            print(f'{regionFinalAcc} -> {regionToTest} added                                           ')
        else:
            regionsTrainData[region].remove(regionToTest)
            
        #print(f'[{nn+1} of {len(regionsToTest)}] testing: {regionToTest}                        ',end='\r')
    
    regionsStats['accuracy_0'][n] = regionFinalAcc
    regionsStats['size'][n] = len(pred)
    
    print(f'\n\naccuracy: {regionFinalAcc}')
    print(f'{region} train data: {regionsTrainData[region]}\nlen:{len(regionsTrainData[region])}')
    print(f'test data len: {len(pred)}\n')
    
printFinalResults(regionsStats, 'accuracy_0')

hide_toggle()


[1 of 5] region LPL -> nan:

0.439 -> MSI added                                           
0.438 -> Demacia added                                           
0.436 -> LCS_Tier2 added                                           
0.434 -> REL added                                           
0.433 -> Dutch added                                           
0.429 -> NA_Tier2 added                                           
0.428 -> EBL added                                           
0.424 -> PCS added                                           



KeyboardInterrupt



In [12]:
print(np.mean(regionsStats['accuracy_0']))
print(np.mean(regionsStats['accuracy_1']))
print(np.mean(regionsStats['accuracy_2']))
regionsStats

nan
nan
nan


,region,model,size,accuracy_0,accuracy_1,accuracy_2,cut_off_var
0,LPL,0,NaN,NaN,NaN,NaN,1.5
1,LCK,0,NaN,NaN,NaN,NaN,1.5
2,PCS,0,NaN,NaN,NaN,NaN,1.5
3,VCS,0,NaN,NaN,NaN,NaN,1.5
4,Ultraliga,0,NaN,NaN,NaN,NaN,1.5


### FEATURE SELECTION

In [13]:
%%time

regionsStats['accuracy_1'] = np.nan
regionsFeatureCols = dict(zip(regionsList,[0]*len(regionsList)))

for key in regionsFeatureCols:
    regionsFeatureCols[key] = featureCols.copy()

for n,region in enumerate(regionsToPredict):
    print('=========\n')
    regionFinalAcc = regionsStats['accuracy_0'][n]
    tempTournamentId = region+currentSemesterYear
    cut_off_var = regionsStats['cut_off_var'][n]
    region_model_number = regionsStats['model'][n]
    print(f'[{n+1} of {len(regionsToPredict)}] region {region} -> {regionFinalAcc}:\n')
    print(f'model: {region_model_number}')
    
    initialFeatures = regionsFeatureCols[region].copy()
    for nn,feature in enumerate(initialFeatures):
        regionsFeatureCols[region].remove(feature)
    
        metric, pred = generateMetric(region_model_number, regionsTrainData[region], regionsFeatureCols[region]
                                      , cut_off_var, tempTournamentId, currentTarget, dfToSplit, splitType)
        if metric < regionFinalAcc or np.isnan(regionFinalAcc):
            regionFinalAcc = metric
            print(f'{feature} removed for {metric}                                                ')
        else:
            regionsFeatureCols[region].append(feature)
            
        print(f'[{nn+1} of {len(initialFeatures)}] testing: {feature}                        ',end='\r')
    
    regionsStats['accuracy_1'][n] = regionFinalAcc
    print(f'\n\naccuracy: {regionFinalAcc}')
    print(f'{region} feature count: {len(regionsFeatureCols[region])}')
    print(f'test data len: {len(pred)}\n')
    
printFinalResults(regionsStats, 'accuracy_1')

hide_toggle()


[1 of 5] region LPL -> nan:

model: 0


KeyboardInterrupt: 

### MODEL SELECTION

In [10]:
%%time

regionsStats['accuracy_2'] = np.nan

for n,region in enumerate(regionsToPredict):
    print('=========\n')
    currAcc = regionsStats['accuracy_0'][n]
    currModel = regionsStats['model'][n]
    tempTournamentId = region+currentSemesterYear
    cut_off_var = regionsStats['cut_off_var'][n]
    print(f'[{n+1} of {len(regionsToPredict)}] region {region} -> {currAcc}:\n')
    print(f'current model: {currModel}\n')
    
    bestModelAbs = (regionsStats[regionsStats['region']==region])['accuracy_2'].iloc[0]
    for model in range(len(base_models)):
        metricModelAbs, pred = generateMetric(model, regionsTrainData[region], regionsFeatureCols[region]
                                      , cut_off_var, tempTournamentId, currentTarget, dfToSplit, splitType)
        if metricModelAbs<bestModelAbs or np.isnan(bestModelAbs):
            bestModelAbs=metricModelAbs
            bestModel=model
        print(f'model {model} -> {metricModelAbs}')

    regionsStats['model'][n] = bestModel
    regionsStats['accuracy_2'][n] = bestModelAbs
    
    print(f'\naccuracy: {bestModelAbs}')
    print(f'best model: {bestModel}\n')
    
printFinalResults(regionsStats, 'accuracy_2')

hide_toggle()


[1 of 21] region LPL -> 0.388:

current model: 4

model 0 -> 0.328
model 1 -> 0.34
model 2 -> 0.483
model 3 -> 0.299
model 4 -> 0.403
model 5 -> 0.395
model 6 -> 0.488
model 7 -> 0.322

accuracy: 0.299
best model: 3


[2 of 21] region LCK -> 0.321:

current model: 4

model 0 -> 0.333
model 1 -> 0.282
model 2 -> 0.435
model 3 -> 0.319
model 4 -> 0.354
model 5 -> 0.359
model 6 -> 0.391
model 7 -> 0.319

accuracy: 0.282
best model: 1


[3 of 21] region PCS -> 0.337:

current model: 4

model 0 -> 0.354
model 1 -> 0.324
model 2 -> 0.343
model 3 -> 0.298
model 4 -> 0.386
model 5 -> 0.448
model 6 -> 0.465
model 7 -> 0.305

accuracy: 0.298
best model: 3


[4 of 21] region VCS -> 0.404:

current model: 4

model 0 -> 0.458
model 1 -> 0.348
model 2 -> 0.5
model 3 -> 0.402
model 4 -> 0.482
model 5 -> 0.536
model 6 -> 0.375
model 7 -> 0.424

accuracy: 0.348
best model: 1


[5 of 21] region Ultraliga -> 0.366:

current model: 4

model 0 -> 0.364
model 1 -> 0.528
model 2 -> 0.528
model 3 -> 0.37
mod

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0

In [14]:
print(np.mean(regionsStats['accuracy_0']))
print(np.mean(regionsStats['accuracy_1']))
print(np.mean(regionsStats['accuracy_2']))
regionsStats

nan
nan
nan


,region,model,size,accuracy_0,accuracy_1,accuracy_2,cut_off_var
0,LPL,0,NaN,NaN,NaN,NaN,1.5
1,LCK,0,NaN,NaN,NaN,NaN,1.5
2,PCS,0,NaN,NaN,NaN,NaN,1.5
3,VCS,0,NaN,NaN,NaN,NaN,1.5
4,Ultraliga,0,NaN,NaN,NaN,NaN,1.5


In [15]:
print(np.mean(regionsStats['accuracy_0']))
print(np.mean(regionsStats['accuracy_1']))
print(np.mean(regionsStats['accuracy_2']))
regionsStats

nan
nan
nan


,region,model,size,accuracy_0,accuracy_1,accuracy_2,cut_off_var
0,LPL,0,NaN,NaN,NaN,NaN,1.5
1,LCK,0,NaN,NaN,NaN,NaN,1.5
2,PCS,0,NaN,NaN,NaN,NaN,1.5
3,VCS,0,NaN,NaN,NaN,NaN,1.5
4,Ultraliga,0,NaN,NaN,NaN,NaN,1.5


# Notes

In [9]:
### DROP OUTLIERS

%%time

# regionsFeatureCols = dict(zip(regionsList,[0]*len(regionsList)))
# for key in regionsFeatureCols:
#     regionsFeatureCols[key] = featureCols.copy()

# for n,region in enumerate(regionsToPredict):
#     print('=========\n')
#     regionFinalAcc = regionsStats['accuracy_0'][n]
#     tempTournamentId = region+currentSemesterYear
#     cut_off_var = regionsStats['cut_off_var'][n]
#     region_model_number = regionsStats['model'][n]
#     print(f'[{n+1} of {len(regionsToPredict)}] region {region} -> {regionFinalAcc}:\n')
#     print(f'current var: {cut_off_var}')
#     for var in np.arange(1.0,2.0,0.1):
        
#         metric, pred = generateMetric(region_model_number, regionsTrainData[region], regionsFeatureCols[region]
#                                       , var, tempTournamentId, currentTarget, dfToSplit, splitType)
        
#         print(f'var: {round(var,2)}, metric: {metric}')
#         if metric < regionFinalAcc or np.isnan(regionFinalAcc):
#             regionFinalAcc = metric
#             regionsStats['cut_off_var'][n] = var
#             print(f'changed to {round(var,2)} cut-off for {metric}                                                ')
            
#         #print(f'[{nn+1} of {len(initialFeatures)}] testing: {feature}                        ',end='\r')
    
#     regionsStats['accuracy_0'][n] = regionFinalAcc
#     print(f'\n\naccuracy: {regionFinalAcc}')
#     print(f'test data len: {len(pred)}\n')
    
# printFinalResults(regionsStats, 'accuracy_0')

hide_toggle()


[1 of 5] region LPL -> 0.402:

current var: 1.5


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0